<a href="https://colab.research.google.com/github/pkpk1717/FIFA-19-player-recommender-system/blob/master/FIFA_19_Player_similar_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline


!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import zipfile
import os
import shutil


/bin/sh: pip: command not found


ImportError: No module named 'pydrive'

In [ ]:
## GOOGLE DRIVE AUTHENTICATION
# -codepk03
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
## SEACHING FOR THE ZIP FILE AND THEN UNZIPPING IT IN COLAB AND DELETING UN-NECESSARY FILES

fid = drive.ListFile({'q':"title='fifa19.zip'"}).GetList()[0]['id']
f = drive.CreateFile({'id': fid})
f.GetContentFile('fifa19.zip')
with zipfile.ZipFile("fifa19.zip","r") as zip_ref:
    zip_ref.extractall("fifa_data/")
os.unlink('fifa19.zip')

In [ ]:
data = pd.read_csv('fifa_data/data.csv')

In [ ]:
data.head()

In [ ]:
data.describe()

In [ ]:
data.columns

## Initial insights

In [ ]:
plt.figure(figsize = (15,10))
sns.boxplot(x = "Work Rate", y = "Age", data = data)
plt.show()

In [ ]:
data.isna().sum().sort_values(ascending=False)

Above stats shows that there are a lot of empty values and this large number can not be filled, so dropping the most empty columns woud help.

In [ ]:
data.drop('Loaned From',axis=1,inplace=True)

In [ ]:
data['Nationality'].value_counts()[:15].plot(kind='bar')
plt.show()

Most number of players are from England, Germany and Spain as expected since 3 major leagues games are played in football there.

In [ ]:
## lets drop unnecessary columns

cols_to_be_dropped = ['ID','Unnamed: 0','Weak Foot','Release Clause','Wage','Photo', 'Nationality', 'Flag',
                      'Club Logo', 'International Reputation',
                      'Work Rate', 'Body Type', 'Real Face','Jersey Number', 'Joined', 
                      'Contract Valid Until','LS', 'ST', 'RS', 'LW', 'LF', 'CF', 'RF', 
                      'RW','LAM', 'CAM', 'RAM', 'LM', 'LCM', 'CM', 'RCM', 'RM', 'LWB', 
                      'LDM','CDM', 'RDM', 'RWB', 'LB', 'LCB','CB', 'RCB', 'RB',]
data.drop(cols_to_be_dropped,axis=1,inplace=True)
data.head()

In [ ]:
data['Difference'] = data['Potential']-data['Overall']

def evolution(d):
    if d==0:
        return 'Stable'
    elif d>1 and d<=5:
        return 'Small'
    elif d>6 and d<=10:
        return 'Medium'
    elif d>10:
        return 'Large'
   
 
data['Evolution'] = data['Difference'].apply(evolution)
promising = data.loc[(data['Evolution']== 'Large') & (data['Potential']>85)].sort_values(by='Potential', ascending=False)[:10]

MOST PROMISING YOUTH PLAYER WHO HAVE A VERY HIGH POTENTIAL OF BECOME GREAT ARE :

In [ ]:
cols  = ['Name','Club','Overall','Age','Potential','Value']
promising[cols]

In [ ]:
sns.countplot(data['Preferred Foot'])
plt.title('Player preferred Foot')

In [ ]:
plt.figure(figsize=(15,10))
sns.countplot(data['Position'])
plt.title('Number of players per position')

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA

In [ ]:
# chooosing only import attritubes
data = pd.read_csv('fifa_data/data.csv')

attributes = data.iloc[:, 54:83]
attributes['Skill Moves'] = data['Skill Moves']
workrate = data['Work Rate'].str.get_dummies(sep='/ ')
attributes = pd.concat([attributes, workrate], axis=1)
df = attributes
attributes = attributes.dropna()
df['Name'] = data['Name']
df = df.dropna()
print(attributes.columns)

In [ ]:
attributes.head()

In [ ]:
scaled = StandardScaler()
X = scaled.fit_transform(attributes)

In [ ]:
recommendations = NearestNeighbors(n_neighbors=6,algorithm='ball_tree')
recommendations.fit(X)

In [ ]:
player_index = recommendations.kneighbors(X)[1]

In [ ]:
player_index

In [ ]:
def get_index(x):
    return df[df['Name']==x].index.tolist()[0]

def recommend_me(player):
    print("5 Players similar to {} are : ".format(player))
    index=  get_index(player)
    for i in player_index[index][1:]:
        print(df.iloc[i]['Name'])

In [ ]:
recommend_me('L. Messi')

In [ ]:
recommend_me('Cristiano Ronaldo')

In [ ]:
recommend_me(player='De Gea')

In [ ]:
recommend_me(player='K. De Bruyne')